# Can Congress and Twitter data be linked?

>How to use Congress API: https://github.com/eyeseast/propublica-congress/blob/master/docs/api.rst

In [129]:
import tweepy
import pandas as pd

## Set up Congress API: Senate

In [130]:
from congress import Congress

congress = Congress('qa1qfTzGinGGLcNCbinp2OTlI2SvzBe4KFgxYobm')

# get member by bioguide ID
senate = congress.members.filter('senate')

In [131]:
#access members in dict
sen_members = senate[0]['members']

#save in df
df_senate = pd.DataFrame(sen_members)
df_senate.columns #twitter_account gives twitter name

Index(['api_uri', 'contact_form', 'crp_id', 'cspan_id', 'date_of_birth',
       'dw_nominate', 'facebook_account', 'fax', 'fec_candidate_id',
       'first_name', 'gender', 'google_entity_id', 'govtrack_id', 'icpsr_id',
       'id', 'ideal_point', 'in_office', 'last_name', 'leadership_role',
       'lis_id', 'middle_name', 'missed_votes', 'missed_votes_pct',
       'next_election', 'ocd_id', 'office', 'party', 'phone', 'rss_url',
       'senate_class', 'seniority', 'short_title', 'state', 'state_rank',
       'suffix', 'title', 'total_present', 'total_votes', 'twitter_account',
       'url', 'votes_with_party_pct', 'votesmart_id', 'youtube_account'],
      dtype='object')

In [132]:
total = df_senate.shape[0]

In [133]:
print ("Share of Senators without Twitter account:", (df_senate.twitter_account.isnull().sum()/total)*100,'\n'
      "Share of Senators without YouTube account:", (df_senate.youtube_account.isnull().sum()/total)*100
      )

Share of Senators without Twitter account: 3.8461538461538463 
Share of Senators without YouTube account: 15.384615384615385


>Good News: only 3% do not have a twitter account

## Set up Tweepy

In [134]:
from tweepy import OAuthHandler
from tweepy import API
from tweepy import Cursor
from datetime import datetime, date, time, timedelta
from collections import Counter
import sys

In [135]:
consumer_key= 'AKEpYmXTTBDu6lePXsBfOz3zy'
consumer_secret = 'h6Kz7HPfmGVCVqsKeck6kSwOsIxnIHeMpEy6NOUkfSYHjF8NMJ'
access_token="991768520546488321-OvBfDZg6YEZpo3oWnabsOj5rhRBCNvY"
access_token_secret="wLJUj2hDB0wE4A4x2TgNnesEqFmqHbyRlNvMorWjcoMfw"
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
auth_api = API(auth)

In [136]:
#save account names in list an skip 'None'
account_list = [i for i in df_senate['twitter_account'] if not i == None] 
account_list[:3]

['SenAlexander', 'SenatorBaldwin', 'SenJohnBarrasso']

In [141]:
#return account age in days
def account_age(name):
    if not name == None: 
        user = auth_api.get_user(name)
        account_created_date = user.created_at
        delta = datetime.utcnow() - account_created_date 
        return delta.days
    else:
        return None

#average tweets per day
def tweets_dagen(name):
    if not name == None: 
        user = auth_api.get_user(name)
        tweets = user.statuses_count
        return (tweets/account_age(name))
    else:
        return None

In [138]:
#to select those with an account
df_senate['twitter_avail'] = df_senate.twitter_account.notnull()

In [140]:
df_senate['tweets_dagen'] = df_senate.twitter_account[df_senate.twitter_avail == True].apply(tweets_dagen) 

TweepError: [{'code': 50, 'message': 'User not found.'}]

In [ ]:
df_senate.tweets_dagen.sort_values(ascending=True)

In [ ]:
df_senate.iloc[:10,-4:]

#### Infos about followers

In [105]:
def get_follower_ids(target):
    return auth_api.followers_ids(target)

In [110]:
len(get_follower_ids(account_list[0])) #somehow stopping at 5000!

5000

>fetches only the most recent 5000 followers